# Homework 1

In [1]:
import pandas as pd 
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

In [2]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')

In [3]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [4]:
len(df)

1154112

## Q2. Computing duration

In [5]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
df.describe()

,PUlocationID,DOlocationID,duration
count,195845.000000,991892.000000,1.154112e+06
mean,139.859690,135.898030,1.916722e+01
std,74.991382,80.474902,3.986922e+02
min,1.000000,1.000000,1.666667e-02
25%,75.000000,67.000000,7.766667e+00
50%,143.000000,132.000000,1.340000e+01
75%,206.000000,213.000000,2.228333e+01
max,265.000000,265.000000,4.233710e+05


In [7]:
df.duration.mean()

19.1672240937939

## Q3. Missing values

In [9]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [10]:
len(df)

1109826

In [11]:
features = ["PUlocationID", "DOlocationID"]
X_train = df[features]

In [12]:
X_train.isna().sum() / len(X_train)

PUlocationID    0.835273
DOlocationID    0.133270
dtype: float64

## Q4. One-hot encoding

In [13]:
X_train = X_train.fillna(-1)
X_train = X_train.astype(str)

In [14]:
train_dicts = X_train.to_dict(orient="records")

In [15]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [16]:
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [18]:
y_train = df["duration"]

## Q5. Training a model

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

## Q6. Evaluating the model

In [20]:
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

10.5285191072072

In [21]:
def eval_(fn, dv, lr):
    df = pd.read_parquet(fn)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    features = ["PUlocationID", "DOlocationID"]
    X = df[features]
    
    X = X.fillna(-1)
    X = X.astype(str)
    val_dicts = X.to_dict(orient="records")

    X_val = dv.transform(val_dicts)
    y_val = df["duration"]

    y_pred = lr.predict(X_val)
    loss = mean_squared_error(y_val, y_pred, squared=False)
    
    return loss

In [22]:
eval_("./data/fhv_tripdata_2021-02.parquet", dv, lr)

11.014283163400654